In [73]:
import os
import random
import numpy as np
import pandas as pd
from math import ceil
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import decomposition
from statistics import mode
#import vggish_input, vggish_slim, vggish_params, utils
#from utils import wavefile_to_waveform

In [3]:
speaker_id = ['1069','289','39','6181','8226','1334','3242','412','6272','831',
              '1553','332','5163','7402','8465','1737','3436','6000','7800',
              '8609','2159','3440','6064','7859','8770','2384','3807','6081',
              '8095','8975']

In [4]:
len(speaker_id)

30

In [14]:
# construct an df column = [emb, true_label]
a = np.array([1, 2, 3])
label_a = '1'
b = np.array([4, 5, 6])
label_b = '4'

np.concatenate((a,b), axis = 0)
#pd.DataFrame({'emb': a.T, 'label': label_a})

array([1, 2, 3, 4, 5, 6])

In [27]:
emb = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/6064-56168-0016.npy')

In [32]:
len(emb[0])

6144

In [30]:
emb.shape[0] # how many split for each audio file = length
emb.shape[1] # = 6144

153

In [6]:
np.concatenate((a, b), axis=0)

array([1, 2, 3, 4, 5, 6])

In [79]:
#train
import os

directory = r'/Users/ruoyuzhu/30-speakers/train/embeddings_6144/'


first = True
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        
        if first:
            emb = np.load(entry.path) #load array 
            audio_size = emb.shape[0]  # how many 1s clips in each file
            true_label = entry.path.split('/')[6].split('-')[0] #replace 3 by path number
            X = emb
            y = np.repeat(true_label, audio_size)
            print(y.shape)
            print(X.shape)
            first = False
        else:
            emb = np.load(entry.path)
            audio_size = emb.shape[0]
            true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
            X = np.concatenate((X, emb))
            next_label = np.repeat(true_label, audio_size)
            y = np.concatenate((y, next_label))
            
            

        

(121,)
(121, 6144)


In [80]:
import pandas as pd 
pd.DataFrame(X).to_csv("X.csv")
pd.DataFrame(y).to_csv("y.csv")

In [82]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)
print('Fitting model..')

Fitting model..


In [92]:
#test
directory = r'/Users/ruoyuzhu/30-speakers/test/embeddings_6144/'

y_true = []
y_pred = []
first = True
for entry in os.scandir(directory):
    if (entry.path.endswith(".npy") and entry.is_file()):
        emb_test = np.load(entry.path)
        #audio_size = emb.shape[0]
        
        print(entry.path)
        print(emb_test)
        
        true_label = entry.path.split('/')[6].split('-')[0]  #label type = 'string'
        predict_label = mode(clf.predict(emb_test))
        print(y_true, y_pred)
        y_true.append(true_label)
        y_pred.append(predict_label)
            


/Users/ruoyuzhu/30-speakers/test/embeddings_6144/6081-42010-0008.npy
[[ 0.66436315 -0.1995123   1.0730408  ...  0.5112637   1.1984068
   2.3605049 ]
 [ 0.8303027  -0.08987964  1.1508265  ...  0.50920975  1.1954324
   2.3194153 ]
 [ 0.6084308   0.00655666  0.95224935 ...  0.5101579   1.1943808
   2.415895  ]
 ...
 [ 1.5204761  -0.09172014  1.1079652  ...  0.50865966  1.2012653
   2.708792  ]
 [ 1.7198637  -0.3170219   1.28633    ...  0.5111443   1.2018423
   2.753244  ]
 [ 1.839895   -0.31760922  1.121341   ...  0.53159136  1.219121
   2.724468  ]]
[] []
/Users/ruoyuzhu/30-speakers/test/embeddings_6144/3242-8112-0023.npy
[[ 0.256846   -0.153609    1.4874564  ...  0.55743784  1.2120464
   2.765602  ]
 [ 0.21593647 -0.03757725  1.29233    ...  0.5109865   1.2026721
   2.659474  ]
 [ 0.19066533 -0.18450576  1.0869292  ...  0.5450424   1.2062048
   2.5003874 ]
 ...
 [ 1.0763222  -0.0104371   1.0958781  ...  0.5331745   1.207733
   2.6519    ]
 [ 0.8798693  -0.1764392   0.6492531  ...  0.530

KeyboardInterrupt: 

In [86]:
# print and store the results
conf_matrix = confusion_matrix(y_true, y_pred)
acc = accuracy_score(y_true, y_pred)

In [87]:
conf_matrix

array([[16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 18,  

In [88]:
acc

1.0

In [98]:
# save the model to disk
import pickle
filename = 'baseline_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(emb_test) # '6064'
print(result)

['2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159' '2159'
 '2159' '2159' '2159' '2159' '2159' '2159']


In [94]:
len(emb[0])

6144

In [5]:
#DATA_FOLDER = '/Users/ruoyuzhu/sklearn-audio-transfer-learning/data/'  #'../data/'
DATA_FOLDER = '/Users/ruoyuzhu/30-speakers/
config = {
    'dataset': 'LibriSpeech-360',
    'num_classes_dataset': 30, #30 speakers
    'audio_folder': DATA_FOLDER + 'audio/GTZAN/genres/',
    'audio_paths_train': DATA_FOLDER + 'index/GTZAN/train_filtered.txt',
    'audio_paths_test': DATA_FOLDER + 'index/GTZAN/test_filtered.txt',
    'batch_size': 8, 
    'features_type': 'openl3', 
    'pca': False, 
    'model_type': 'RandomForest',
    'load_training_data': False, 
    'load_evaluation_data': False 
}

In [6]:
## import function wavefile_to_waveform
import os
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io import wavfile


# def load_path2gt(paths_file, config):
#     """ Given the path, construct the ground truth vectors.
#         This function heavily relies on path2gt_datasets(.),
#         where the relation between the path and ground truth
#         are defined.
#     """
#     paths = list()
#     path2gt = dict()
#     path2onehot = dict() # REMOVE IF NOT USED!
#     pf = open(paths_file)
#     for path in pf.readlines():
#         path = path.rstrip('\n')
#         paths.append(path)
#         label = path2gt_datasets(path, config['dataset'])
#         path2gt[path] = label
#         path2onehot[path] = label2onehot(label, config['num_classes_dataset'])  #array([0., 0., 1., 0., 0.])
#     return paths, path2gt, path2onehot


# def label2onehot(label, num_classes):
#     """ Convert class label to one hot vector.
#         Example: label2onehot(label=2, num_classes=5) > array([0., 0., 1., 0., 0.])
#     """
#     onehot = np.zeros(num_classes)
#     onehot[label] = 1
#     return onehot



In [77]:
emb = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/6064-56168-0016.npy')

In [46]:
emb2 = np.load('/Users/ruoyuzhu/30-speakers/train/embeddings_6144/3436-172162-0003.npy')

In [52]:
X = np.concatenate((emb, emb2))
Y = np.repeat(label, audio_size)

array([[ 1.0185577 , -0.18536666,  1.0069084 , ...,  0.58884275,
         1.2111325 ,  2.0921428 ],
       [ 1.1984097 , -0.17363879,  0.9581679 , ...,  0.5749884 ,
         1.2113444 ,  2.2201946 ],
       [ 1.127706  , -0.15025726,  0.77957195, ...,  0.51264477,
         1.1968786 ,  2.2820084 ],
       ...,
       [ 0.88844043,  0.0154366 ,  0.5938695 , ...,  0.5180066 ,
         1.1988028 ,  2.3797178 ],
       [ 0.7915392 ,  0.06348451,  0.5508625 , ...,  0.5322705 ,
         1.1990378 ,  2.5066764 ],
       [ 0.9272685 , -0.11993069,  0.5081603 , ...,  0.4983559 ,
         1.2113365 ,  2.5066237 ]], dtype=float32)

array(['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'],
      dtype='<U1')